In [2]:
from crewai import Task, Agent, Crew
from utils.utils import get_openai_api_key
import os
from pathlib import Path

os.environ["OPENAI_API_KEY"] = get_openai_api_key()
os.environ["MODEL"] = "gpt-4o-mini"

In [3]:
# import the tools
from crewai_tools import EXASearchTool, ScrapeWebsiteTool
from utils.utils import get_exa_api_key



In [4]:
# set the exa API key
os.environ["EXA_API_KEY"] = get_exa_api_key()

In [5]:
# Create the EXASearchTool instance
exa_search_tool = EXASearchTool(base_url=os.getenv("EXA_BASE_URL"))
# Create the ScrapeWebsiteTool instance
scrape_website_tool = ScrapeWebsiteTool()

In [6]:
# agente creation: ResearchAgent
research_planner = Agent(
    role="Research Planner",
    goal="Analyze queries and break them down into smaller, specific research topics.",
    backstory=(
         "You are a research strategist who excels at breaking down complex questions "
         "into manageable research components. You identify what needs to be researched "
         "and create clear research objectives."
    ),
    verbose=True, # set to True to see detailed agent actions
    max_rpm=150,
    max_iter=15
)

In [7]:
# remain agents creation: researcher, fact_checker, report_writer

researcher = Agent(
    role="Internet Researcher",
    goal="Research thoroughly all assigned topics",
    ### START CODE HERE ###
    backstory=(
        "You are a skilled researcher with experience in online investigation "
        "and data collection. You know how to find reliable sources, extract relevant information, "
        "and always verify facts across multiple sources to avoid misinformation or hallucination. "
        "You never invent facts and always trace information to its origin."
    ),
    # add the 2 tool instances you created
    tools=[exa_search_tool, scrape_website_tool],
    ### END CODE HERE ###
    verbose=True,
    max_rpm=150,
    max_iter=15
)

fact_checker = Agent(
    role="Fact Checker",
    goal=(
        "Verify data for accuracy, identify inconsistencies, "
        "and flag potential misinformation"
    ),
    ### START CODE HERE ###
    backstory=(
        "You are a quality assurance specialist with expertise in fact-checking "
        "and identifying misinformation and hallucinations. You cross-reference information, "
        "spot inconsistencies, and ensure all data meets high accuracy standards. You rigorously"
        "check for hallucinated or invented content and require that all facts be supported"
        "by evidence."
    ),
    tools=[exa_search_tool, scrape_website_tool],
    ### END CODE HERE ###
    verbose=True,
    max_rpm=150,
    max_iter=15
)

report_writer = Agent(
    role="Report Writer",
    goal="Write clear, concise, and well-structured reports based on gathered information",
    ### START CODE HERE ###
    backstory=(
         "You are an expert writer who specializes in creating clear, well-structured "
        "research reports. You synthesize complex information into readable formats and "
        "always include proper citations and sources."
    ),
    ### END CODE HERE ###
    verbose=True,max_rpm=150,
    max_iter=15
)
          

In [8]:
# task creation

# define the create research plan task
create_research_plan_task = Task(
    description=(
        "Based on the user's query, break it down into specific topics and key questions, "
        "and create a focused research plan."
        "The user's query is: {user_query}"
    ),
    expected_output=(
        "A research plan with main research topics to investigate, "
        "key questions for each topic, and success criteria for the research."
        ),
    agent=research_planner,
)

In [9]:
# define the gather research data task
gather_research_data_task = Task(
    description=(
        "Using the research plan, collect information on all identified topics. "
        "Cite all sources used."
    ),
    ### START CODE HERE ###
    expected_output=(
        "Comprehensive research data including: information for each"
        "research topic, and citations used along with source credibility notes"
    ),
    agent=researcher,
    ### END CODE HERE ###
)

#define the verify information quality task
verify_information_quality_task = Task(
    description=(
        "Review all collected research. Identify any conflicting information, "
        "potential misinformation, or gaps that need addressing."
    ),
    ### START CODE HERE ###
    expected_output=(
        "A report with all the original data you got plus any "
        "verified facts vs. questionable information, make sure this is as comprehensive "
        "as possible for final report generation"
    ),
    agent=fact_checker,
    ### END CODE HERE ###
)

# define the write final report task
write_final_report_task = Task(
    description=(
        "Create a comprehensive report that answers the original query using all verified research data. "
        "Structure it with clear sections, include citations, and provide actionable insights."
    ),
    ### START CODE HERE ###
    expected_output=(
        "A final research report containing: executive summary, detailed "
        "findings that answer the user query, supporting evidence and analysis, complete "
        "source citations."
    ),
    agent=report_writer,
    ### END CODE HERE ###
)
    

In [10]:
# create the crew with the defined agents and tasks
crew = Crew(
    ### START CODE HERE ###
    agents=[research_planner, researcher, fact_checker, report_writer],
    tasks=[create_research_plan_task, gather_research_data_task, verify_information_quality_task, write_final_report_task],
    ### 
)

In [11]:
### START CODE HERE ###

# Write your query, which will be used as input for the tasks.
user_query = "Evaluate the top five emerging AI tools for automating competitive market analysis, including their features, limitations, costs, and ideal use cases for a mid-sized marketing firm."

### END CODE HERE ###

In [ ]:
result = crew.kickoff(
    inputs={
        "user_query": user_query,
    }
)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Planner                                                                                        │
│                                                                                                                 │
│  Task: Based on the user's query, break it down into specific topics and key questions, and create a focused    │
│  research plan.The user's query is: Evaluate the top five emerging AI tools for automating competitive market   │
│  analysis, including their features, limitations, costs, and ideal use cases for a mid-sized marketing firm.    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Planner                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Research Plan: Evaluating Top Five Emerging AI Tools for Automating Competitive Market Analysis**            │
│                                                                                                                 │
│  **Main Research Topics:**                                                                                      │
│                                                                                                                 │
│  1. **Identification of Top Five AI Tools**                                                                     │
│     - **Key Questions:**                                                                                        │
│       - What are the current leading AI tools in the market for automating competitive market analysis?         │
│       - How are these tools differentiated in terms of technology and functionality?                            │
│       - What sources provide recent trustworthy data regarding these tools?                                     │
│     - **Success Criteria:**                                                                                     │
│       - A list of five AI tools along with a brief overview of each, including their market positioning and     │
│  technology.                                                                                                    │
│                                                                                                                 │
│  2. **Features of Each AI Tool**                                                                                │
│     - **Key Questions:**                                                                                        │
│       - What specific features do these AI tools offer for competitive market analysis?                         │
│       - How do the tools utilize AI technologies (e.g., machine learning, natural language processing)?         │
│       - Are there integration capabilities with other marketing or CRM tools?                                   │
│     - **Success Criteria:**                                                                                     │
│       - A detailed comparison chart showcasing the features of the five tools side-by-side.                     │
│                                                                                                                 │
│  3. **Limitations and Challenges**                                                                              │
│     - **Key Questions:**                                                                                        │
│       - What are the known limitations or challenges associated with each tool?                                 │
│       - How do these limitations impact their effectiveness in specific scenarios?                              │
│       - Are there user reviews or case studies that highlight these limitations?                                │
│     - **Success Criteria:**                                                                                     │
│       - A comprehensive analysis detailing the limitations of each tool, including user feedback where          │
│  applicable.                                                                                                    │
│                                                        

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Internet Researcher                                                                                     │
│                                                                                                                 │
│  Task: Using the research plan, collect information on all identified topics. Cite all sources used.            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Internet Researcher                                                                                     │
│                                                                                                                 │
│  Thought: I need to start by identifying the top five AI tools that are currently leading in the market for     │
│  automating competitive market analysis. This requires current and reliable data regarding these tools, their   │
│  technology, and functionality.                                                                                 │
│                                                                                                                 │
│  Using Tool: EXASearchTool                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "search_query": "top AI tools for automating competitive market analysis 2023",                              │
│    "start_published_date": "2022-01-01",                                                                        │
│    "end_published_date": "2023-10-31",                                                                          │
│    "include_domains": null                                                                                      │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Fact Checker                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### Comprehensive Overview of Research Findings on Emerging AI Tools for Competitive Market Analysis           │
│                                                                                                                 │
│  **1. Brandwatch Consumer Intelligence**                                                                        │
│  - **Overview**: Brandwatch is recognized as a leading consumer intelligence tool designed for social media     │
│  monitoring and competitive market analysis.                                                                    │
│  - **Market Positioning**: Established market presence since 2007, especially post-acquisition by Cision.       │
│  - **Technology**: AI for advanced analytics and data visualization.                                            │
│  - **Source**: Influencer Marketing Hub.                                                                        │
│  - **Strengths**: Strong for sentiment analysis and trend tracking.                                             │
│  - **Limitations**: Can have an overwhelming number of data filters.                                            │
│                                                                                                                 │
│  **2. SEMrush**                                                                                                 │
│  - **Overview**: An extensive digital marketing tool that offers features for competitor analysis,              │
│  particularly in SEO and PPC.                                                                                   │
│  - **Market Positioning**: Widely adopted by digital marketers.                                                 │
│  - **Technology**: User-friendly interface with robust integration capabilities.                                │
│  - **Source**: SEMrush Blog.                                                                                    │
│  - **Strengths**: Strong on traffic analytics and advertising research.                                         │
│  - **Limitations**: Pricing may be high for smaller enterprises.                                                │
│                                                                                                                 │
│  **3. Similarweb**                                                                                              │
│  - **Overview**: A tool providing insights into website traffic, user behavior, and competitive benchmarking.   │
│  - **Market Positioning**: A comprehensive platform for digital market research.                                │
│  - **Technology**: Diverse data sources and user interface designed for usability.                              │
│  - **Source**: SEMrush Blog.                                                                                    │
│  - **Strengths**: Accurately benchmarks competitors across many digital channels.                               │
│  - **Limitations**: Some criticisms regarding data accuracy in smaller markets.                                 │
│                                                                                                                 │
│  **4. Kompyte**                                                                                                 │
│  - **Overview**: Automates competitive intelligence dat

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Report Writer                                                                                           │
│                                                                                                                 │
│  Task: Create a comprehensive report that answers the original query using all verified research data.          │
│  Structure it with clear sections, include citations, and provide actionable insights.                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Report Writer                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Comprehensive Report on Emerging AI Tools for Automating Competitive Market Analysis                         │
│                                                                                                                 │
│  ## Executive Summary                                                                                           │
│                                                                                                                 │
│  The landscape of competitive market analysis is rapidly evolving due to the adoption of artificial             │
│  intelligence (AI) tools. This report evaluates five leading AI tools that assist mid-sized marketing firms in  │
│  automating their competitive market analysis processes. The tools analyzed in this report include Brandwatch   │
│  Consumer Intelligence, SEMrush, Similarweb, Kompyte, and Ahrefs. Each tool's key features, limitations,        │
│  pricing models, and ideal use cases are discussed in detail to provide actionable insights for                 │
│  decision-makers in mid-sized marketing firms. Comprehensive comparisons and up-to-date references are          │
│  included to facilitate informed choices in the selection of these AI tools.                                    │
│                                                                                                                 │
│  ## 1. Identification of Top Five AI Tools                                                                      │
│                                                                                                                 │
│  ### 1.1 Brandwatch Consumer Intelligence                                                                       │
│  - **Overview**: Brandwatch is a powerful consumer intelligence tool used for social media monitoring, market   │
│  research, and competitive analysis.                                                                            │
│  - **Market Positioning**: Established in 2007 and acquired by Cision in 2021, it is primarily aimed at medium  │
│  and enterprise companies.                                                                                      │
│  - **Technology**: Employs AI for advanced data analytics and visualization capabilities.                       │
│  - **Source**: [Influencer Marketing Hub](https://influencermarketinghub.com/competitive-intelligence-tools/)   │
│  - **Credibility**: Known for actionable insights and a long-standing reputation in the analytics field.        │
│                                                                                                                 │
│  ### 1.2 SEMrush                                                                                                │
│  - **Overview**: SEMrush is a comprehensive tool for digital marketing that integrates functionalities for      │
│  competitor analysis in SEO and PPC.                                                                            │
│  - **Market Positioning**: It is widely used by marketers to enhance their own strategies based on competitive  │
│  data.                                                                                                          │
│  - **Technology**: Features a user-friendly interface with capabilities for integration with various marketing  │
│  tools.                                                



╭────────────────────────────── Execution Traces ──────────────────────────────╮
│                                                                              │
│  🔍 Detailed execution traces are available!                                 │
│                                                                              │
│  View insights including:                                                    │
│    • Agent decision-making process                                           │
│    • Task execution flow and timing                                          │
│    • Tool usage details                                                      │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯
Would you like to view your execution traces? [y/N] (20s timeout): 



╭────────────────────────── Tracing Preference Saved ──────────────────────────╮
│                                                                              │
│  Info: Tracing has been disabled.                                            │
│                                                                              │
│  Your preference has been saved. Future Crew/Flow executions will not        │
│  collect traces.                                                             │
│                                                                              │
│  To enable tracing later, do any one of these:                               │
│  • Set tracing=True in your Crew/Flow code                                   │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file               │
│  • Run: crewai traces enable                                                 │
│                                                                              │
╰─────────────────────────

In [13]:
from IPython.display import Markdown
Markdown(result.raw) 

# Comprehensive Report on Emerging AI Tools for Automating Competitive Market Analysis

## Executive Summary

The landscape of competitive market analysis is rapidly evolving due to the adoption of artificial intelligence (AI) tools. This report evaluates five leading AI tools that assist mid-sized marketing firms in automating their competitive market analysis processes. The tools analyzed in this report include Brandwatch Consumer Intelligence, SEMrush, Similarweb, Kompyte, and Ahrefs. Each tool's key features, limitations, pricing models, and ideal use cases are discussed in detail to provide actionable insights for decision-makers in mid-sized marketing firms. Comprehensive comparisons and up-to-date references are included to facilitate informed choices in the selection of these AI tools.

## 1. Identification of Top Five AI Tools

### 1.1 Brandwatch Consumer Intelligence
- **Overview**: Brandwatch is a powerful consumer intelligence tool used for social media monitoring, market research, and competitive analysis.
- **Market Positioning**: Established in 2007 and acquired by Cision in 2021, it is primarily aimed at medium and enterprise companies.
- **Technology**: Employs AI for advanced data analytics and visualization capabilities. 
- **Source**: [Influencer Marketing Hub](https://influencermarketinghub.com/competitive-intelligence-tools/)
- **Credibility**: Known for actionable insights and a long-standing reputation in the analytics field.

### 1.2 SEMrush
- **Overview**: SEMrush is a comprehensive tool for digital marketing that integrates functionalities for competitor analysis in SEO and PPC.
- **Market Positioning**: It is widely used by marketers to enhance their own strategies based on competitive data.
- **Technology**: Features a user-friendly interface with capabilities for integration with various marketing tools.
- **Source**: [SEMrush Blog](https://www.semrush.com/blog/best-competitive-intelligence-tools/)
- **Credibility**: Well-regarded among digital marketing professionals.

### 1.3 Similarweb
- **Overview**: Similarweb provides insights into competitor website traffic and overall market behavior.
- **Market Positioning**: Functions as an all-in-one digital market research tool for businesses.
- **Technology**: Utilizes a diverse array of data sources.
- **Source**: [SEMrush Blog](https://www.semrush.com/blog/best-competitive-intelligence-tools/)
- **Credibility**: Known for accuracy in data and rich market insights.

### 1.4 Kompyte
- **Overview**: Automates competitive intelligence by tracking competitors across various platforms, aiding teams in sales and marketing.
- **Market Positioning**: Designed to enhance speed and efficiency in market analysis.
- **Technology**: Leverages machine learning for real-time data gathering.
- **Source**: [Influencer Marketing Hub](https://influencermarketinghub.com/competitive-intelligence-tools/)
- **Credibility**: Noted for its innovative data visualization features.

### 1.5 Ahrefs
- **Overview**: Predominantly an SEO tool, Ahrefs also includes extensive competitive analysis functions such as backlink checking.
- **Market Positioning**: Utilized extensively by SEO experts and digital marketers.
- **Technology**: Powered by substantial databases for competitive insights.
- **Source**: [SEMrush Blog](https://www.semrush.com/blog/best-competitive-intelligence-tools/)
- **Credibility**: Renowned in the SEO community for robust analytics.

## 2. Features Comparison Chart

| Tool                | Key Features                                               | Technology Utilization               | User Base              |
|---------------------|-----------------------------------------------------------|--------------------------------------|------------------------|
| Brandwatch          | Social monitoring, sentiment analysis, trend analysis      | AI-driven analytics                  | Medium to Enterprise    |
| SEMrush             | Traffic Analytics, SEO tools, Social Media Tracker        | Integrated marketing capabilities     | Broad marketing teams   |
| Similarweb          | Traffic insights, benchmarking, market research           | Comprehensive data collection        | Businesses of all sizes |
| Kompyte             | Real-time tracking, automated updates                      | Machine learning-driven insights     | Sales and marketing teams|
| Ahrefs              | Backlink analysis, keyword tracking                        | Rich SEO tools                       | Digital marketers       |

## 3. Limitations and Challenges

### 3.1 Brandwatch
- **Limitations**: Some users report that the extensive data filters can be overwhelming and may require a learning curve to navigate effectively.

### 3.2 SEMrush
- **Limitations**: The high pricing tier can restrict smaller businesses from accessing essential features. Additionally, real-time data updates may be limited compared to competitors.

### 3.3 Similarweb
- **Limitations**: Critics mention occasional discrepancies in data, particularly in smaller or emerging markets, which can impact the insights derived.

### 3.4 Kompyte
- **Limitations**: Its reliance on AI may lead to occasional inaccuracies in classifying competitor changes, necessitating human oversight for interpretation in certain instances.

### 3.5 Ahrefs
- **Limitations**: While excelling in SEO analysis, Ahrefs may fall short in providing comprehensive data insights across broader marketing functions.

## 4. Cost Analysis

| Tool                | Pricing Structure                     | Notes on Additional Costs            |
|---------------------|---------------------------------------|--------------------------------------|
| Brandwatch          | Pricing on request; considered premium| Custom implementations may result in added fees |
| SEMrush             | Starting at $129.95/month            | Enterprise solutions add significant costs |
| Similarweb          | Custom pricing; typically premium     | Potentially high costs for advanced features |
| Kompyte             | Starting around $99/month             | Additional charges for premium features |
| Ahrefs              | Starting at $99/month                 | Costs scale with feature access      |

## 5. Ideal Use Cases for Mid-Sized Marketing Firms

### 5.1 Brandwatch
- **Best For**: Firms focusing on brand health and social media sentiment analysis to guide marketing strategy.

### 5.2 SEMrush
- **Best For**: Businesses concentrated on improving their SEO and PPC strategies, with a focus on tracking performance against competitors.

### 5.3 Similarweb
- **Best For**: Mid-sized firms requiring deep insights into traffic analytics to better understand their competitive landscape.

### 5.4 Kompyte
- **Best For**: Marketing and sales teams needing fast updates and competitive tracking to adjust strategies promptly.

### 5.5 Ahrefs
- **Best For**: SEO-centric teams looking for detailed backlinks and keyword performance comparatives.

## Recommendations
Mid-sized marketing firms should consider **Kompyte** for quick data access and **SEMrush** for comprehensive analysis in SEO and PPC as they provide functionalities that cater to immediate market research needs.

## Sources
- Influencer Marketing Hub: [Top Competitive Intelligence Tools](https://influencermarketinghub.com/competitive-intelligence-tools/)
- SEMrush: [Best Competitive Intelligence Tools](https://www.semrush.com/blog/best-competitive-intelligence-tools/)

This comprehensive analysis equips mid-sized marketing firms with essential insights needed to make well-informed decisions regarding AI tools for competitive market analysis. The report highlights the evolution of market analysis tools and the importance of integrating AI for dynamic decision-making in an increasingly competitive landscape.